# Temporal-Difference (TD) Learning

## Background:

**Value function**
It would be great to know how **good** a given state s is. Something to tell us: no matter the state we are in if we transition to state s your total reward will be x, word! If you start from s and follow policy π. That would spare us from revisiting same states over and over again. The value function does this for us. It depends on the state we are in s and the policy π your agent is following. It is given by:

<img src="td_data/v_func.png">


There exists an optimal value function that has the highest value for all states. It is given by:

<img src="td_data/v_opt.png">

**Q function**
Agent can not control what state it ends up in, directly. It can influence enviornment by choosing some action **a**. Let us introduce another function that accepts state and action as parameters and returns the expected total reward — the Q function (it represents the **quality** of a certain action given a state). More formally, the function Q^π(s, a) gives the expected return when starting in state **s**, performing action **a** and following π.

Again, we can define the optimal Q-function Q∗(s, a) that gives the expected total reward for your agent when starting at s and picks action a. That is, the optimal Q-function tells your agent how good of a choice is picking a when at state s.

There is a relationship between the two optimal functions V∗ and Q∗. It is given by:

<img src="td_data/v_and_q.png">

That is, the maximum expected total reward when starting at s is the maximum of Q∗(s, a) over all possible actions.

Using Q∗(s, a) we can extract the optimal policy π∗ by choosing the action aa that gives maximum reward Q∗(s, a) for state s. We have:

<img src="td_data/pi.png">

## TD Introduction

- TD is a combination of ideas from Monto-Carlo methods and DP methods
- TD methods learn directly from episodes of raw experience without a model, like MC
- TD methods update estimates based in part on other learned estimates, without waiting for a final outcome (i.e. they bootstrap), like DP
- TD updates a guess towards a guess
- General Update Rule: Q[s,a] += learning_rate * (td_target - Q[s,a]); where as (td_target - Q[s,a]) is called the TD Error
- TD Methods: 
- - Evaluation/Prediction: TD(0), TD(lambda)
- - Control: SARSA, Q-Learning

<img src="td_data/dp_td.png">

# TD(0) - TD Policy Evaluation 

Policy Evaluation focus on estimating the Vπ - for a given policy π, compute the value function Vπ.

**Bellman Expectation Equation for Vπ(s):**
<img src="td_data/bellman_exp.png">

**Hyper-Parameters:**

- **alpha:** learning rate (0 to 1) is weight given to the new information versus the old information. A factor 0 makes the agent learn nothing; while a factor of 1 makes the agent consider only the most recent information.


- **lambda:** discount factor (0 to 1) to penalize future rewards compared to the immediate reward.

<img src="td_data/td_eval_algo.png">

## Create Gridworld Enviornment

<img src="td_data/grid.png">

In [1]:
%matplotlib inline

import gym
import itertools
import matplotlib
import numpy as np
import pandas as pd
import sys
import random
from collections import defaultdict

if "../" not in sys.path:
  sys.path.append("../")

In [2]:
import numpy as np
# Let's start by being able to run episodes in an environment.

from gym.spaces import Discrete

class DiscreteEnvironment:
    """
    Discrete states, discrete actions. We assume rewards on actions. Formally, this means:
    R_t = r(S_t, A_t). "The reward is a function of the state and the action we're in". Alternative formalizations
    include rewards on states alone, and rewards depending on the next state as well.
    """
    class TransitionMatrix:
        def __init__(self, environment, dynamics):
            self.env = environment
            self.dynamics = dynamics
            if callable(dynamics):
                # We assume that a function takes (state, action)
                self.transition_by_function = True
            else:
                self.transition_by_function = False
                if self.dynamics.shape != (len(self.env.states), len(self.env.actions), len(self.env.states)):
                    raise Exception("Transition matrix should be n_states per n_actions per n_states")

        def transition(self, state, action):
            if self.transition_by_function:
                return self.dynamics(state, action)
            else:
                return np.random.choice(self.env.states, p=self.dynamics[state, action])[0]

    def __init__(self, n_states, terminal_states, n_actions, transition_matrix, rewards):
        self.state_space = Discrete(n_states)
        self.terminal_states = np.array(terminal_states)
        self.nonterminal_states = [x for x in range(self.state_space.n) if x not in self.terminal_states]
        self.action_space = Discrete(n_actions)
        self.transition_matrix = DiscreteEnvironment.TransitionMatrix(self, transition_matrix)
        self.rewards = np.array(rewards)
        self.cur_state = np.random.choice(self.nonterminal_states)

    def step(self, action):
        next_state = self.transition_matrix.transition(self.cur_state, action)
        reward = self.rewards[self.cur_state, action]
        self.cur_state = next_state
        self.done = self.cur_state in self.terminal_states
        return self.cur_state, reward, self.done

    def reset(self):
        self.cur_state = np.random.choice(self.nonterminal_states)
        self.done = False
        return self.cur_state

    # Ok, let's implement a Builder pattern

    class Builder:
        def __init__(self):
            self._n_states = None
            self._n_actions = None

            self._terminal_states = None

            self._transition_dynamics = None
            self._rewards = None

        def set_n_states(self, n_states):
            self._n_states = n_states
            return self

        def set_terminal_states(self, list_of_terminals):
            self._terminal_states = list(list_of_terminals)
            return self

        def set_n_actions(self, n_actions):
            self._n_actions = n_actions
            return self

        def set_transition_dynamics(self, transitions):
            self._transition_dynamics = transitions
            return self

        def set_rewards(self, rewards):
            self._rewards = rewards
            return self

        def build(self):
            return DiscreteEnvironment(
                self._n_states,
                self._terminal_states,
                self._n_actions,
                self._transition_dynamics,
                self._rewards
            )

# Create Policy

In [3]:
import numpy as np
from gym.spaces import Discrete, Box # Let's deal with those for now.

class Policy:
    def __init__(self, state_space, action_space):
        pass

    def step(self, state):
        pass


class DiscretePolicy(Policy):
    """
    This is the first implementation of a Policy abstraction that I've done in my life.

    """
    def __init__(self, state_space, action_space):
        super(DiscretePolicy, self).__init__(state_space, action_space)
        self.policy = np.ones((state_space.n, action_space.n)) * (1 / action_space.n)
        self.action_space = action_space
        self.state_space = state_space

    def step(self, state):
        return np.random.choice(np.arange(self.action_space.n, dtype=np.int), p=self.policy[state])


class ContinuousStatePolicy(Policy):
    def __init__(self, state_space, action_space):
        super(ContinuousStatePolicy, self).__init__(state_space, action_space)
        self.action_space = action_space
        self.state_space = state_space

    def step(self, state):
        return self.action_space.sample()

## Monte-Carlo Algorithm

In [4]:
"""
The idea is to implement TD and MonteCarlo methods to evaluate policies
"""
import numpy as np

class MonteCarlo:
   
    def _run_episode(env, policy, with_actions):
        done = False
        cur_st = env.reset()
        rewards = [0.0]
        visited_states = []
        while not done:
            action = policy.step(cur_st)
            if with_actions:
                visited_states.append((cur_st, action))
            else:
                visited_states.append(cur_st)
            new_st, reward, done, *_ = env.step(action)
            rewards.append(reward)
            cur_st = new_st
        return visited_states, rewards

   
    def state_value_eval(env, policy,
                         discount=0.999,
                         learning_rate=0.01,
                         n_iter=1000,
                         print_every=None):
        """
        This is EVERY-VISIT Monte-Carlo
        :param env: An Environment that we can reset(), step() and get observations and
                    reward information.
        :param policy: A strategy for behaving in an Environment. Should have a step()
                    method that returns an action given state information.
        :param discount: Discount factor for the MDP
        :param learning_rate: The amount we will shift towards an error direction.
        :param n_iter: Number of episodes to run this algorithm for
        :param print_every: Print the current estimate of values every X iterations
        :return: The State-Value function that shows the average return we'll have starting
                 in each one of the states of this MDP
        """
        state_values = [0.0 for _ in range(env.state_space.n)]

        for episode in range(n_iter):
            visited_states, rewards = MonteCarlo._run_episode(env, policy, with_actions=False)
            for i, state in enumerate(visited_states):
                if i + 1 >= len(rewards):
                    break
                    
                discounted_return_from_state = np.dot(np.array(rewards[i + 1:]),
                           np.fromfunction(lambda i: discount ** i, ((len(rewards) - i - 1),)))
                
                state_values[state] += learning_rate * (discounted_return_from_state - state_values[state])
                
            if print_every is not None and episode % print_every == 0:
                print('State-Value estimation:\n{}'.format(state_values))
                
        return state_values   




## TD(0) Algorithm

In [8]:
class TDZero:
   
    def state_value_eval(env, policy,
                         discount=0.999, learning_rate=0.01,
                         n_iter=1000, print_every=None):
        state_values = [0.0 for _ in range(env.state_space.n)]

        for episode in range(n_iter):
            done = False
            cur_state = env.reset()
            while not done:
                action = policy.step(cur_state)
                new_st, reward, done, *_ = env.step(action)
                state_values[cur_state] += learning_rate * (reward + discount * state_values[new_st] - state_values[cur_state])
                cur_state = new_st
                
            if print_every is not None and episode % print_every == 0:
                print('State-Value estimation:\n{}'.format(state_values))
                
        return state_values    

# TD(lambda) Algorithm

In [9]:
class TDLambda:  
    
    def state_value_eval(env, policy, lamb,
            discount=0.999,
            learning_rate=0.01,
            n_iter=1000,
            print_every=None):
        state_values = [0.0 for _ in range(env.state_space.n)]
        for episode in range(n_iter):
            eligibility = [0.0 for _ in range(env.state_space.n)]
            done = False
            cur_state = env.reset()
            while not done:
                for s in range(env.state_space.n):
                    eligibility[s] *= lamb * discount
                eligibility[cur_state] += 1
                action = policy.step(cur_state)
                new_st, reward, done, *_ = env.step(action)
                error = reward + state_values[new_st] - state_values[cur_state]
                for s in range(env.state_space.n):
                    state_values[s] += learning_rate * eligibility[s] * error
                    
                cur_state = new_st
                
            if print_every != None and episode % print_every == 0:
                print('Action-Value estimation:\n{}'.format(state_values))
        return state_values

## Test and Compare MC, TD(0) and TD(lambda)

In [10]:

import numpy as np
from gym.spaces import Box, Discrete

import gym

import pprint

pp = pprint.PrettyPrinter(indent=2)


def simple_dynamics(state, action):
    if action == 0:
        new_state = state - 1
    else:
        new_state = state + 1
    return new_state


def rewards(state: int, action: int) -> int:
    return int(simple_dynamics(state, action) == 6)


def rewards_builder(rows, cols):
    return np.array(
        [rewards(*pair) for pair in zip(rows.ravel(), cols.ravel())]
    ).reshape(rows.shape)


env = DiscreteEnvironment.Builder()\
    .set_n_actions(2)\
    .set_n_states(7)\
    .set_transition_dynamics(simple_dynamics)\
    .set_rewards(np.fromfunction(rewards_builder, (7, 2)))\
    .set_terminal_states([0, 6])\
    .build()

policy = DiscretePolicy(Discrete(7), Discrete(2))

print('Evaluate using Monte Carlo!')
# from policy_evaluation import monte_carlo_eval

state_val = MonteCarlo.state_value_eval(env, policy)

print('Monte Carlo Results:')
print('State values:\n{}'.format(
    dict(enumerate(state_val))
))

state_val = TDZero.state_value_eval(env, policy)

print('TD(0) Results:')
print('State values:\n{}'.format(
    dict(enumerate(state_val))
))

lamb = 0.9
state_val = TDLambda.state_value_eval(env, policy, lamb)

print('TD(lambda={:.3f}) Results:'.format(lamb))
print('State values:\n{}'.format(
    dict(enumerate(state_val))
))


Evaluate using Monte Carlo!
Monte Carlo Results:
State values:
{0: 0.0, 1: 0.2185337835631336, 2: 0.41634866445170965, 3: 0.61335097709002018, 4: 0.77763945743722518, 5: 0.87312916538502083, 6: 0.0}
TD(0) Results:
State values:
{0: 0.0, 1: 0.10882314926460014, 2: 0.2278201004612776, 3: 0.4004911070166528, 4: 0.58361299145799783, 5: 0.7890644637231442, 6: 0.0}
TD(lambda=0.900) Results:
State values:
{0: 0.0, 1: 0.15816559161946203, 2: 0.34340546392240284, 3: 0.54979230217080022, 4: 0.74041925387006846, 5: 0.87951848509006958, 6: 0.0}
